In [134]:
# import requirements 
import urllib2
import zipfile 
import pandas as pd 
from __future__ import division

## Download and read the dataset 

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"

In [3]:
response = urllib2.urlopen(url)

In [4]:
spam_data = response.read()

In [5]:
with open("smsspamcollection.zip",'w') as f:
    f.write(spam_data)

In [6]:
!ls

Spam_detection.ipynb  smsspamcollection     smsspamcollection.zip


In [7]:
z = zipfile.ZipFile('smsspamcollection.zip','r')

In [8]:
for filename in z.namelist():
    print filename
    SMSSpamCollection = z.read(filename)

SMSSpamCollection
readme


In [9]:
SMSSpamCollection_formatted = pd.read_table("SMSspamCollection/SMSSpamCollection",
                                     sep = "\t",
                                     header = None, 
                                     names = ['label','sms_message']) 

In [124]:
SMSSpamCollection_formatted.head()

,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


### Initial EDA 

In [128]:
print SMSSpamCollection_formatted.groupby('label').count()

       sms_message
label             
0             4825
1              747


In [130]:
print 'Total number of spams in the data set are ', sum(SMSSpamCollection_formatted.label)

Total number of spams in the data set are  747


In [158]:
print 'Per%  of spams in the data set would be :',
round(sum(SMSSpamCollection_formatted.label)/len(SMSSpamCollection_formatted),3)*100 

 Per%  of spams in the data set would be :

13.4

In [11]:
# we need to convert the labels to a one hot encoding format for building the model 
SMSSpamCollection_formatted["label"] = SMSSpamCollection_formatted.label.map({"ham":0,"spam":1})

In [13]:
print SMSSpamCollection_formatted.shape
SMSSpamCollection_formatted.head()

(5572, 2)


,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


## Model Building 
### 1. Naive Bayes classifier

In [22]:
#perform the test train split on our dataset 
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(SMSSpamCollection_formatted.sms_message,
                                                 SMSSpamCollection_formatted.label,
                                                test_size = 0.33,random_state = 1)

In [47]:
print "The number of rows in the total data is",len(SMSSpamCollection)
print "The number of rows in the training data is",len(X_train)
print "The number of rows in the test data is",len(Y_test)

The number of rows in the total data is 5868
The number of rows in the training data is 3733
The number of rows in the test data is 1839


In [15]:
## Implementing Bag of words using TFIDF vectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
#instantiate the model 
tfidfVec = TfidfVectorizer()
print tfidfVec

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)


In [62]:
#fit the model 
training_data = tfidfVec.fit_transform(X_train)

In [63]:
# Transform testing data and return the matrix. Note we are not fitting the testing data into the tfidfvectorizer
test_data = tfidfVec.transform(X_test)

In [64]:
#using naive Bayes model
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data,Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [65]:
training_data

<3733x6984 sparse matrix of type '<type 'numpy.float64'>'
	with 49152 stored elements in Compressed Sparse Row format>

In [66]:
predictions = naive_bayes.predict(test_data)

In [82]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
print "Accuracy Score:" , format(round(accuracy_score(predictions,Y_test),3))
print "Precision Score:" , format(round(precision_score(predictions,Y_test),3))
print "recall Score:" , format(round(recall_score(predictions,Y_test),3))
print "f1 Score:" , format(round(f1_score(predictions,Y_test),3))


Accuracy Score: 0.955
Precision Score: 0.669
recall Score: 1.0
f1 Score: 0.802


In [86]:
# we can get pretty much the same metrics by class using sklearn.metrics.classification_report module 
from sklearn.metrics import classification_report
report = classification_report(predictions,Y_test)
print report

             precision    recall  f1-score   support

          0       1.00      0.95      0.97      1673
          1       0.67      1.00      0.80       166

avg / total       0.97      0.96      0.96      1839



### 2. Random forest model 

In [91]:
from sklearn.ensemble import RandomForestClassifier
#instantiate the random forest object 
RF_model = RandomForestClassifier()
#fit the model 
RF_model.fit(training_data,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [98]:
#using the model to make predictions 
rf_predictions = RF_model.predict(test_data)
print "Accuracy Score:" , format(round(accuracy_score(rf_predictions,Y_test),3))
print "so our accuracy score improved by around:" ,0.018

Accuracy Score: 0.968
so our accuracy score improved by around: 0.018


In [94]:
report_rf  = classification_report(rf_predictions,Y_test)
print report_rf

             precision    recall  f1-score   support

          0       1.00      0.97      0.98      1645
          1       0.77      0.99      0.87       194

avg / total       0.98      0.97      0.97      1839



### 3. Multilevel Perceptron in sklearn

In [104]:
from sklearn.neural_network import MLPClassifier
#instantiate the model 
MLP_class = MLPClassifier()
# fit the model 
MLP_class.fit(training_data,Y_train)
# make predictions 
predictions_mlp = MLP_class.predict(test_data)

In [105]:
print "MLP model accuracy is :",accuracy_score(predictions_mlp,Y_test)

MLP model accuracy is : 0.988580750408


In [107]:
report_mlp = classification_report(predictions_mlp,Y_test)
print report_mlp

             precision    recall  f1-score   support

          0       1.00      0.99      0.99      1608
          1       0.92      0.99      0.96       231

avg / total       0.99      0.99      0.99      1839



Conclusion: It looks like we are getting pretty high accuracy,precision and recall with the Multilevel perceptron model here without even having to tune our model parameters. 

"A naive Bayes classifier considers each of these features to contribute independently to the probability that this fruit is an apple, regardless of any possible correlations between the color, roundness, and diameter features."
source :wiki

So our naive bayes classifier would have over simplified the problem.